# Linear Regression with sklearn and seaborn

For this excersise I have chosen a dataset from [Kaggle](https://www.kaggle.com/) reflecting a timeseries of financial trading data on [Ethereum-Classic-ETC](https://www.kaggle.com/soham1024/ethereum-historical-data) crypto-currency. This dataset was updated Aug 2, 2020
at the time of this analysis and covers historical data at 1 day intervals for the period beginning Mar 2016 to Aug 02, 2020. 

The goal is to Identify:

```
    A) Does a correlation exist between one days' Close Price (shown as price in the dataset) and the previous days' volume. 
    B) If such a correlation exists, can it be used to create a predictive model of the next days' Price? High? Low? Spread?

```

For the purpose of this analysis, Volume will be considered as our constant variable. (should no direct correlations appear to exist with any other category, Volume as a ratio of one other column series will be considered as our next constant)

The Dependent Variable for this analysis will be set to Price(on the subsequent day of the constant)



In [44]:
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


#TODO: run script that alters csv string representations of floats containing comma's. call below function while iterating through each line. O(N^2)
# def pull_comma(csv_str):
#     if ',' in csv_str:
#         return csv_str.pop(csv_str[','])
#     else:
#         return csv_str

In [65]:
df_etc = pd.read_csv('data_csv/Ethereum Historical Data.csv')
df_etc['Date'] = pd.to_datetime(df_etc['Date'])
# df_etc = df_etc.convert_dtypes() # only use if necessary to perform operations on values in DataFrame

#TODO: if pandas cannot work with the string values execute below to_numeric conversions after running script to alter cvs file in above cell.
# pd.to_numeric(df_etc['Price'])
# pd.to_numeric(df_etc['Open'])
# pd.to_numeric(df_etc['High'])
# pd.to_numeric(df_etc['Low'])
# pd.to_numeric(df_etc['Vol'])
# pd.to_numeric(df_etc['Change %'])

df_etc.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1607 entries, 0 to 1606
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      1607 non-null   datetime64[ns]
 1   Price     1607 non-null   object        
 2   Open      1607 non-null   object        
 3   High      1607 non-null   object        
 4   Low       1607 non-null   object        
 5   Vol.      1607 non-null   object        
 6   Change %  1607 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 88.0+ KB


In [69]:

df_etc.describe()



,Date,Price,Open,High,Low,Vol.,Change %
count,1607,1607,1607,1607,1607,1607,1607
unique,1607,1499,1492,1490,1482,1376,1014
top,2019-04-29 00:00:00,10.16,10.16,10.16,10.16,-,0.00%
freq,1,9,8,7,7,8,14
first,2016-03-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
last,2020-08-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


### Setting constant, dependent, train_test_split, and Instantiating LinearRegression



In [ ]:
#TODO: account for the fact that in this time/series evaluation the value of the Constant:(X) is one day behind the value being considered for the Dependent:(y). potential solve. drop first indexrow from Dependent:(y) and drop last indexrow from Constant:(X) this will offset the data being considered without affecting the shape of the data. 

X = df_etc['Vol.'].values 
y = df_etc['Price'].values
